In [ ]:
from tensorflow import keras
keras.__version__

In [ ]:
from tensorflow.keras.datasets import boston_housing

(train_data0, train_targets), (test_data0, test_targets) =  boston_housing.load_data()

In [ ]:
train_data0.shape

In [ ]:
test_data0.shape

In [ ]:
# 타깃은 주택의 중간 가격으로 천달러 단위.

# 13개의 특성:
# 
# 1. Per capita crime rate. : 1인당 범죄율
# 2. Proportion of residential land zoned for lots over 25,000 square feet. : 주거용 토지비율
# 3. Proportion of non-retail business acres per town. : 비상업용 토지 비율
# 4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise). : 강변 여부
# 5. Nitric oxides concentration (parts per 10 million). : 산화질소 농도
# 6. Average number of rooms per dwelling. : 가구당 방수
# 7. Proportion of owner-occupied units built prior to 1940. 1940년 이전 건축되고 주인이 거주하는 비율
# 8. Weighted distances to five Boston employment centres.: 보스턴 고용센터까지의 거리
# 9. Index of accessibility to radial highways. : 고속도로 접근지수
# 10. Full-value property-tax rate per $10,000. : 
# 11. Pupil-teacher ratio by town. : 학생:교사 비율
# 12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town. : 흑인비율 관련
# 13. % lower status of the population. : 


In [ ]:
train_targets[:10]

## 데이터 준비 - Scaling
#### - 모든 데이타를 평균0, 표준편차 1로 정리.

In [ ]:
import numpy as np

train_data = np.copy(train_data0)
test_data = np.copy(test_data0)

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# 테스트 데이터를 정규화할 때 사용한 값이 훈련 데이터에서 계산한 값이어야 함. -->
test_data -= mean
test_data /= std

In [ ]:
train_data.mean(axis=0), train_data.std(axis=0)

In [ ]:
# sklearn의 StandardScaler를 사용해서 검증해보면...
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_data0)
train_data = scaler.transform(train_data0)
test_data = scaler.transform(test_data0)

In [ ]:
train_data.mean(axis=0), train_data.std(axis=0)
# test_data.mean(axis=0), test_data.std(axis=0)

## 모델 구성

샘플의 개수가 적기 때문에 64개의 유닛을 가진 두 개의 은닉층으로 작은 네트워크를 구성

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    # 동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## K-겹 검증을 사용한 훈련 검증


### epoch = 50 적용.
#### 4폴드로 나누고 4번 model fit 호출

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 50
all_mae_histories = []

for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

에포크마다 모든 폴드의 MAE 평균 계산.

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

* 곡선의 다른 부분과 스케일이 많이 다른 첫 10개 데이터 포인트 제외.
* 부드러운 곡선을 얻기 위해 이동 평균으로 대체.

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

검증 MAE에서 최소가 되는 epoch 확인 --> 이후 더 학습하면 과대적합.

In [ ]:
# 새롭게 컴파일된 모델을 얻습니다
model = build_model()
# 전체 데이터로 훈련시킵니다
model.fit(train_data, train_targets,
          epochs=40, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

test_mae_score

이 지점이 최상의 성능을 내는가?  
최상의 성능을 내는 epoch 값을 어떻게 찾을까?

In [ ]:
np.argmin()